<a href="https://colab.research.google.com/github/pedroconcejero/UTAD_python/blob/master/huggingface_summarisation_con_TF_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets evaluate rouge_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 17.4 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=a9b30f8bf51af0b601e78ce6062289ffde5de751f04509f999be9afecebb32f1
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled hu

In [2]:
from datasets import load_dataset

billsum = load_dataset("billsum", split="ca_test")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/18949 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3269 [00:00<?, ? examples/s]

Generating ca_test split:   0%|          | 0/1237 [00:00<?, ? examples/s]

In [3]:
billsum = billsum.train_test_split(test_size=0.2)

In [4]:
billsum["train"][0]

{'text': 'The people of the State of California do enact as follows:\n\n\nSECTION 1.\nSection 19620.2 of the Business and Professions Code is amended to read:\n19620.2.\n(a) (1) Any unallocated balance from Section 19620.1 and any revenue deposited in the Fair and Exposition Fund pursuant to Section 7101.4 of the Revenue and Taxation Code is hereby appropriated without regard to fiscal years for allocation by the Secretary of Food and Agriculture for capital outlay to California fairs for fair projects involving public health and safety, for fair projects involving major and deferred maintenance, for fair projects necessary due to any emergency, for projects that are required by physical changes to the fair site, for projects that are required to protect the fair property or installation, such as fencing and flood protection, and for the acquisition or improvement of any property or facility that will serve to enhance the operation of the fair.\n(2) Any revenues deposited into the Fair

There are two fields that you’ll want to use:

    text: the text of the bill which’ll be the input to the model.
    summary: a condensed version of text which’ll be the model target.

## Preprocess

The next step is to load a T5 tokenizer to process text and summary:

The preprocessing function you want to create needs to:

    Prefix the input with a prompt so T5 knows this is a summarization task. Some models capable of multiple NLP tasks require prompting for specific tasks.
    Use the keyword text_target argument when tokenizing labels.
    Truncate sequences to be no longer than the maximum length set by the max_length parameter.

In [5]:
from transformers import AutoTokenizer

checkpoint = "google-t5/t5-small"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [6]:
prefix = "summarize: "

# IMPORTANT CHANGED max_length inputs to 512 and of labels to 64.
# OTHERWISE WE HAVE NO RESOURCES

def preprocess_function(examples):

    inputs = [prefix + doc for doc in examples["text"]]

    model_inputs = tokenizer(inputs,
                             max_length=512,
                             truncation=True)

    labels = tokenizer(text_target=examples["summary"],
                       max_length=64,
                       truncation=True)

    model_inputs["labels"] = labels["input_ids"]

    return model_inputs

To apply the preprocessing function over the entire dataset, use 🤗 Datasets map method. You can speed up the map function by setting batched=True to process multiple elements of the dataset at once:

In [7]:
tokenized_billsum = billsum.map(preprocess_function, batched=True)

Map:   0%|          | 0/989 [00:00<?, ? examples/s]

Map:   0%|          | 0/248 [00:00<?, ? examples/s]

Now create a batch of examples using DataCollatorForSeq2Seq. It’s more efficient to dynamically pad the sentences to the longest length in a batch during collation, instead of padding the whole dataset to the maximum length.

In [8]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

# Evaluate

Including a metric during training is often helpful for evaluating your model’s performance. You can quickly load a evaluation method with the 🤗 Evaluate library. For this task, load the ROUGE metric (see the 🤗 Evaluate quick tour to learn more about how to load and compute a metric):

In [9]:
import evaluate

rouge = evaluate.load("rouge")

Then create a function that passes your predictions and labels to compute to calculate the ROUGE metric:

In [10]:
import numpy as np


def compute_metrics(eval_pred):

    predictions, labels = eval_pred

    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)

    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]

    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

## Train Pytorch

You’re ready to start training your model now! Load T5 with AutoModelForSeq2SeqLM:

In [ ]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)



At this point, only three steps remain:

1.     Define your training hyperparameters in Seq2SeqTrainingArguments. The only required parameter is output_dir which specifies where to save your model. You’ll push this model to the Hub by setting push_to_hub=True (you need to be signed in to Hugging Face to upload your model). At the end of each epoch, the Trainer will evaluate the ROUGE metric and save the training checkpoint.
2.     Pass the training arguments to Seq2SeqTrainer along with the model, dataset, tokenizer, data collator, and compute_metrics function.
3.     Call train() to finetune your model.

In [ ]:
training_args = Seq2SeqTrainingArguments(

    output_dir="my_awesome_billsum_model",

    evaluation_strategy="epoch",

    learning_rate=2e-5,

    per_device_train_batch_size=16,

    per_device_eval_batch_size=16,

    weight_decay=0.01,

    save_total_limit=3,

    num_train_epochs=4,

    predict_with_generate=True,

    fp16=True,

    push_to_hub=True,

)

trainer = Seq2SeqTrainer(

    model=model,

    args=training_args,

    train_dataset=tokenized_billsum["train"],

    eval_dataset=tokenized_billsum["test"],

    tokenizer=tokenizer,

    data_collator=data_collator,

    compute_metrics=compute_metrics,

)

trainer.train()

## Train Tensorflow

To finetune a model in TensorFlow, start by setting up an optimizer function, learning rate schedule, and some training hyperparameters:

In [14]:
from transformers import create_optimizer, AdamWeightDecay

optimizer = AdamWeightDecay(learning_rate=2e-5, weight_decay_rate=0.01)

In [15]:
from transformers import TFAutoModelForSeq2SeqLM

model = TFAutoModelForSeq2SeqLM.from_pretrained(checkpoint)

All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


Convert your datasets to the tf.data.Dataset format with prepare_tf_dataset():

In [16]:
tf_train_set = model.prepare_tf_dataset(

    tokenized_billsum["train"],

    shuffle=True,

    batch_size=16,

    collate_fn=data_collator,

)

tf_test_set = model.prepare_tf_dataset(

    tokenized_billsum["test"],

    shuffle=False,

    batch_size=16,

    collate_fn=data_collator,

)

In [17]:
import tensorflow as tf

model.compile(optimizer=optimizer)  # No loss argument!

The last two things to setup before you start training is to compute the ROUGE score from the predictions, and provide a way to push your model to the Hub. Both are done by using Keras callbacks.

Pass your compute_metrics function to KerasMetricCallback:

In [18]:
from transformers.keras_callbacks import KerasMetricCallback

metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=tf_test_set)

In [19]:
callbacks = [metric_callback]

In [21]:
model.fit(x=tf_train_set,
          validation_data=tf_test_set,
          epochs=3)

Epoch 1/3
61/61 [==============================] - 43s 708ms/step - loss: 3.0492 - val_loss: 2.7157
Epoch 2/3
61/61 [==============================] - 43s 699ms/step - loss: 2.9004 - val_loss: 2.6225
Epoch 3/3
61/61 [==============================] - 44s 726ms/step - loss: 2.8172 - val_loss: 2.5606


In [32]:
# Save the weights
model.save('saved_model/my_model')


In [37]:
model.save_pretrained("modelos/modelo1", from_pt=True)

In [33]:
text = "summarize: The Inflation Reduction Act lowers prescription drug costs, health care costs, and energy costs. It's the most aggressive action on tackling the climate crisis in American history, which will lift up American workers and create good-paying, union jobs across the country. It'll lower the deficit and ask the ultra-wealthy and corporations to pay their fair share. And no one making under $400,000 per year will pay a penny more in taxes."

In [27]:
from transformers import pipeline

In [39]:
from transformers import pipeline

pipe = pipeline(task="summarization",
                model='modelos/modelo1',
                tokenizer=tokenizer)

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at modelos/modelo1.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


# Inference - Create summaries

Tokenize the text and return the input_ids as TensorFlow tensors:

In [40]:
pipe(text)

Your max_length is set to 200, but your input_length is only 103. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=51)


[{'summary_text': "the Inflation Reduction Act lowers prescription drug costs, health care costs, and energy costs. It's the most aggressive action on tackling the climate crisis in American history, which will lift up workers and create good-paying, union jobs across the country."}]